In [57]:
import numpy as np
import pandas as pd
import pyodbc as pyo
import sqlalchemy as sa
import numpy as np
import math
import datetime
import datetime as dt
#Kết nối SV_UAT (SQL_SV64)
connection_uri_UAT = (
    "mssql+pyodbc://PHLHCMHOPPBI02\PBIRS/PowerBI?driver=ODBC+Driver+17+for+SQL+Server"
)
conn = sa.create_engine(connection_uri_UAT)

<h3> 1. Lấy bảng

<h5>1.1 Lấy kỳ xét trong quý:

In [63]:
quarter = input("Nhập vào quý xét: YYYYQ: ")
cutoff = []
#Định nghĩa ra các tháng trong quý, modify theo kỳ CUTOFF hàng năm:
quarter_dict = {"1":["0331"], "2":["0630"], "3":["0930"], "4":["1231"]}
for month in quarter_dict[quarter[4:]]:
    cutoff.append(quarter[:4]+ month)
cutoff = cutoff[0]
cutoff

Nhập vào quý xét: YYYYQQ: 20234


'20231231'

In [64]:
premium_query = f"""\n
SELECT * FROM DP_TAGENTPREMIUM_TEST_CUTOFF
WHERE CUTOFF = {cutoff}
"""
premium_table = pd.read_sql(premium_query, conn)

In [104]:
com_query = f"""\n
SELECT * FROM DP_TAGENTCOMM_TEST_CUTOFF
WHERE CUTOFF = {cutoff}
"""
com_table = pd.read_sql(com_query, conn)

In [71]:
premium = premium_table
premium

,Policy No,Product Code,Premium transaction,PREM TRAN NAME,Premium Collected,Collected Date,Applied Premium Date,FYP Before Discount,FYP Discount,FYP,...,Area Code,Servicing Agent,Freelook,Proposal Receive Date,RISK_COMMENCE_DATE,Age_Customer,Issuing Agent,AFYP,CUTOFF,RETURN_DATE
0,80052426,UL01,B522,Premium Collection,5100000.0,2021-04-29,2021-04-29,0.0,0.0,0.0,...,DTH,60008565,1506.0,2019-11-11,2019-11-10,14.0,60022999,13020000.0,20231231,None
1,80052426,URW3,B522,Premium Collection,524000.0,2021-04-29,2021-04-29,524000.0,0.0,524000.0,...,DTH,60008565,1506.0,2019-11-11,2021-01-10,45.0,60022999,13020000.0,20231231,None
2,80052426,UL01,B522,Premium Collection,5100000.0,2021-01-18,2021-01-18,0.0,0.0,0.0,...,DTH,60008565,1506.0,2019-11-11,2019-11-10,14.0,60022999,13020000.0,20231231,None
3,80051270,UL01,B536,Flex Prem Premium Collection,8631000.0,2022-11-22,2022-11-22,0.0,0.0,0.0,...,VIN,60025243,1526.0,2019-10-18,2019-10-18,21.0,60022140,11443000.0,20231231,None
4,80052724,URA1,B522,Premium Collection,282000.0,2022-01-07,2022-01-07,0.0,0.0,0.0,...,HUE,60029079,1460.0,2019-11-20,2019-11-19,23.0,60022753,16680000.0,20231231,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599507,80119902,RHB1,B522,Premium Collection,1040000.0,2023-09-22,2023-09-22,0.0,0.0,0.0,...,VIN,60052037,468.0,2022-09-15,2022-09-15,26.0,60052037,16032000.0,20231231,None
1599508,80119902,URC5,B522,Premium Collection,489000.0,2023-09-22,2023-09-22,0.0,0.0,0.0,...,VIN,60052037,468.0,2022-09-15,2022-09-15,26.0,60052037,16032000.0,20231231,None
1599509,80123240,URT5,B522,Premium Collection,303000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,YTH,60045573,427.0,2022-10-25,2022-10-25,3.0,60045573,15394000.0,20231231,None
1599510,80123240,UL05,B522,Premium Collection,9743000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,YTH,60045573,427.0,2022-10-25,2022-10-25,25.0,60045573,15394000.0,20231231,None


<h3>2. Chỉ lọc ra các Applied Date trong kỳ được xét

<h5>2.1 Lấy ra kỳ được xét (CHÚ Ý CÁCH LẤY NGÀY ĐẦU TIÊN CỦA QUÝ): 

In [85]:
premium['Date_On_Set'] = premium['CUTOFF'].str[:4] + "-" + premium['CUTOFF'].str[4:6] + "-" + premium['CUTOFF'].str[6:]
premium['Date_On_Set'] = pd.to_datetime(premium['Date_On_Set'])
premium['Start_Date_Set'] = premium['Date_On_Set'] - pd.tseries.offsets.QuarterBegin(startingMonth=1)
premium = premium[(premium['Applied Premium Date'] >= premium['Start_Date_Set']) & (premium['Applied Premium Date'] <= premium['Date_On_Set'])]
premium

,Policy No,Product Code,Premium transaction,PREM TRAN NAME,Premium Collected,Collected Date,Applied Premium Date,FYP Before Discount,FYP Discount,FYP,...,Freelook,Proposal Receive Date,RISK_COMMENCE_DATE,Age_Customer,Issuing Agent,AFYP,CUTOFF,RETURN_DATE,Date_On_Set,Start_Date_Set
10898,80128110,RHB1,B522,Premium Collection,520000.0,2023-12-16,2023-12-16,0.0,0.0,0.0,...,369.0,2022-12-13,2022-12-13,30.0,60024513,12200000.0,20231231,None,2023-12-31,2023-10-01
10899,80128110,URC5,B522,Premium Collection,380000.0,2023-12-16,2023-12-16,0.0,0.0,0.0,...,369.0,2022-12-13,2022-12-13,30.0,60024513,12200000.0,20231231,None,2023-12-31,2023-10-01
12295,80068637,UL01,B536,Flex Prem Premium Collection,2702000.0,2023-12-09,2023-12-09,0.0,0.0,0.0,...,1112.0,2020-12-10,2020-12-09,27.0,60023050,18140000.0,20231231,None,2023-12-31,2023-10-01
12296,80065141,UL01,B536,Flex Prem Premium Collection,13736000.0,2023-12-15,2023-12-15,0.0,0.0,0.0,...,1159.0,2020-10-20,2020-10-20,32.0,60031498,17269000.0,20231231,None,2023-12-31,2023-10-01
12297,80065141,URA3,B536,Flex Prem Premium Collection,644000.0,2023-12-15,2023-12-15,0.0,0.0,0.0,...,1159.0,2020-10-20,2020-10-20,1.0,60031498,17269000.0,20231231,None,2023-12-31,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599501,80123240,URT5,B522,Premium Collection,823000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,0.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599502,80123240,URC5,B522,Premium Collection,145000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,25.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599509,80123240,URT5,B522,Premium Collection,303000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,3.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599510,80123240,UL05,B522,Premium Collection,9743000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,25.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01


<h5>2.2 Loại bỏ các HD hủy

In [89]:
premium = premium[~premium['Policy Status'].isin(['CF', 'DC', 'FL', 'NT', 'WD', 'PO'])]
premium

,Policy No,Product Code,Premium transaction,PREM TRAN NAME,Premium Collected,Collected Date,Applied Premium Date,FYP Before Discount,FYP Discount,FYP,...,Freelook,Proposal Receive Date,RISK_COMMENCE_DATE,Age_Customer,Issuing Agent,AFYP,CUTOFF,RETURN_DATE,Date_On_Set,Start_Date_Set
10898,80128110,RHB1,B522,Premium Collection,520000.0,2023-12-16,2023-12-16,0.0,0.0,0.0,...,369.0,2022-12-13,2022-12-13,30.0,60024513,12200000.0,20231231,None,2023-12-31,2023-10-01
10899,80128110,URC5,B522,Premium Collection,380000.0,2023-12-16,2023-12-16,0.0,0.0,0.0,...,369.0,2022-12-13,2022-12-13,30.0,60024513,12200000.0,20231231,None,2023-12-31,2023-10-01
12295,80068637,UL01,B536,Flex Prem Premium Collection,2702000.0,2023-12-09,2023-12-09,0.0,0.0,0.0,...,1112.0,2020-12-10,2020-12-09,27.0,60023050,18140000.0,20231231,None,2023-12-31,2023-10-01
12296,80065141,UL01,B536,Flex Prem Premium Collection,13736000.0,2023-12-15,2023-12-15,0.0,0.0,0.0,...,1159.0,2020-10-20,2020-10-20,32.0,60031498,17269000.0,20231231,None,2023-12-31,2023-10-01
12297,80065141,URA3,B536,Flex Prem Premium Collection,644000.0,2023-12-15,2023-12-15,0.0,0.0,0.0,...,1159.0,2020-10-20,2020-10-20,1.0,60031498,17269000.0,20231231,None,2023-12-31,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1599501,80123240,URT5,B522,Premium Collection,823000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,0.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599502,80123240,URC5,B522,Premium Collection,145000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,25.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599509,80123240,URT5,B522,Premium Collection,303000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,3.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01
1599510,80123240,UL05,B522,Premium Collection,9743000.0,2023-10-26,2023-10-26,0.0,0.0,0.0,...,427.0,2022-10-25,2022-10-25,25.0,60045573,15394000.0,20231231,None,2023-12-31,2023-10-01


<h5>2.3 Loại ra các HD chưa pass 21 ngày FL tại tháng xét cuối cùng của tháng

In [132]:
premium['CheckFL_End'] = np.where((premium['Date_On_Set'] - pd.to_datetime(premium['POLICY ACKNOWLED']))/ pd.offsets.Day(1) >21, "Y", "N")
premium = premium[premium['CheckFL_End'] == "Y"]

<h3>3. Tính FYP của quý

In [133]:
fyp_quarter = premium
fyp_quarter = fyp_quarter.groupby(['Servicing Agent'])[['FYP', 'Topup Premium']].sum().reset_index()
fyp_quarter['FYP_Final'] = fyp_quarter['FYP'] + fyp_quarter['Topup Premium'] * 0.1
fyp_quarter.sort_values('FYP_Final', ascending = False)

,Servicing Agent,FYP,Topup Premium,FYP_Final
6,60000482,40589000.0,0.0,40589000.0
1878,60032592,36025000.0,113000.0,36036300.0
3441,60080155,30767000.0,0.0,30767000.0
3435,60079830,28415000.0,0.0,28415000.0
3383,60074094,27401000.0,0.0,27401000.0
...,...,...,...,...
3009,60059116,0.0,-68600000.0,-6860000.0
3449,60080421,-14986000.0,0.0,-14986000.0
2891,60056101,-16532000.0,0.0,-16532000.0
2396,60045492,-30168000.0,2183000.0,-29949700.0


<h3>4. Tính FYC

In [110]:
com_t = com_table
com_t

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_On_Set,Start_Date_Set
0,80000013,EE01,B522,Premium Collection,3640000.0,2014-12-28,2014-12-28,764000.0,0.0,100.0,...,2014-03-31,2014-03-29,3555.0,2014-03-28,2014-04-08,1,20231231,None,2023-12-31,2023-10-01
1,80000013,EE01,B522,Premium Collection,3640000.0,2015-05-29,2015-05-29,0.0,182000.0,100.0,...,2014-03-31,2014-03-29,3555.0,2014-03-28,2014-04-08,2,20231231,None,2023-12-31,2023-10-01
2,80000013,ETR1,B522,Premium Collection,191000.0,2015-05-29,2015-05-29,0.0,10000.0,100.0,...,2014-03-31,2014-03-29,3555.0,2014-03-28,2014-04-08,2,20231231,None,2023-12-31,2023-10-01
3,80000013,EWR1,B522,Premium Collection,40000.0,2015-05-29,2015-05-29,0.0,2000.0,100.0,...,2014-03-31,2014-03-29,3555.0,2014-03-28,2014-04-08,2,20231231,None,2023-12-31,2023-10-01
4,80000032,RAC2,T642,Contract Issue,540000.0,2014-04-29,2014-04-29,81000.0,0.0,100.0,...,2014-04-29,2014-04-29,3527.0,2014-04-29,2014-05-06,1,20231231,None,2023-12-31,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1576410,80136769,RHB1,T642,Contract Issue,520000.0,2023-03-15,2023-03-15,78000.0,0.0,100.0,...,2023-03-15,2023-03-14,286.0,2023-03-14,2023-03-21,1,20231231,None,2023-12-31,2023-10-01
1576411,80136769,RHB1,T642,Contract Issue,520000.0,2023-03-15,2023-03-15,78000.0,0.0,100.0,...,2023-03-15,2023-03-14,286.0,2023-03-14,2023-03-21,1,20231231,None,2023-12-31,2023-10-01
1576412,80136770,UL06,T642,Contract Issue,48923000.0,2023-03-15,2023-03-15,17612000.0,0.0,100.0,...,2023-03-15,2023-03-14,289.0,2023-03-14,2023-03-18,1,20231231,None,2023-12-31,2023-10-01
1576413,80136770,URA5,T642,Contract Issue,795000.0,2023-03-15,2023-03-15,278000.0,0.0,100.0,...,2023-03-15,2023-03-14,289.0,2023-03-14,2023-03-18,1,20231231,None,2023-12-31,2023-10-01


In [112]:
com_t['Date_On_Set'] = com_t['CUTOFF'].str[:4] + "-" + com_t['CUTOFF'].str[4:6] + "-" + com_t['CUTOFF'].str[6:]
com_t['Date_On_Set'] = pd.to_datetime(com_t['Date_On_Set'])
com_t['Start_Date_Set'] = com_t['Date_On_Set'] - pd.tseries.offsets.QuarterBegin(startingMonth=1)
com_t = com_t[(com_t['Applied Premium Date'] >= com_t['Start_Date_Set']) & (com_t['Applied Premium Date'] <= com_t['Date_On_Set'])]
com_t = com_t[~com_t['Policy Status'].isin(['CF', 'DC', 'FL', 'NT', 'WD', 'PO', 'SU', 'TR'])]
com_t = com_t[com_t['Freelook'].astype('int') >= 21]
com_t

,Policy No,Product Code,Premium transaction,Premium transaction name,Premium Collected,Collected Date,Applied Premium Date,FYC,RYC,Shared Commission Rate,...,Transfer Date,Proposal Receive Date,Freelook,RISK_COMMENCE_DATE,Receive Policy date,Premium Year,CUTOFF,RETURN_DATE,Date_On_Set,Start_Date_Set
41,80000138,EN04,B522,Premium Collection,3020000.0,2023-12-23,2023-12-23,0.0,60000.0,100.0,...,2020-07-13,2014-09-24,3384.0,2014-09-23,2014-09-26,10,20231231,None,2023-12-31,2023-10-01
42,80000138,RTR1,B522,Premium Collection,197000.0,2023-12-23,2023-12-23,0.0,4000.0,100.0,...,2020-07-13,2014-09-24,3384.0,2014-09-23,2014-09-26,10,20231231,None,2023-12-31,2023-10-01
43,80000138,RAC2,B522,Premium Collection,97000.0,2023-12-23,2023-12-23,0.0,3000.0,100.0,...,2020-07-13,2014-09-24,3384.0,2014-09-23,2014-09-26,10,20231231,None,2023-12-31,2023-10-01
50,80000251,EWR1,B522,Premium Collection,40000.0,2023-12-27,2023-12-27,0.0,1000.0,100.0,...,2020-07-13,2014-11-27,3315.0,2014-11-27,2014-12-04,10,20231231,None,2023-12-31,2023-10-01
51,80000251,EE01,B522,Premium Collection,3100000.0,2023-12-27,2023-12-27,0.0,62000.0,100.0,...,2020-07-13,2014-11-27,3315.0,2014-11-27,2014-12-04,10,20231231,None,2023-12-31,2023-10-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1572472,80138742,RHS1,B522,Premium Collection,114000.0,2023-11-28,2023-11-28,23000.0,0.0,100.0,...,2023-03-29,2023-03-29,274.0,2023-03-29,2023-04-02,1,20231231,None,2023-12-31,2023-10-01
1572473,80138742,RHB1,B522,Premium Collection,276000.0,2023-11-28,2023-11-28,41000.0,0.0,100.0,...,2023-03-29,2023-03-29,274.0,2023-03-29,2023-04-02,1,20231231,None,2023-12-31,2023-10-01
1573091,80138301,UL06,B522,Premium Collection,4500000.0,2023-10-06,2023-10-06,1710000.0,0.0,100.0,...,2023-03-27,2023-03-27,278.0,2023-03-27,2023-03-29,1,20231231,None,2023-12-31,2023-10-01
1573093,80138302,UL06,B522,Premium Collection,4500000.0,2023-10-06,2023-10-06,1620000.0,0.0,100.0,...,2023-03-27,2023-03-27,278.0,2023-03-27,2023-03-29,1,20231231,None,2023-12-31,2023-10-01


<h5>4.1 Tính Groupby theo Commission Agent

In [116]:
com_final = com_t
com_final = com_final.groupby(['Commission Agent'])['FYC'].sum().reset_index()
com_final

,Commission Agent,FYC
0,60000194,0.0
1,60000288,0.0
2,60000321,0.0
3,60000401,0.0
4,60000449,0.0
...,...,...
4418,60083473,3447000.0
4419,60083545,1444000.0
4420,60083611,0.0
4421,60083616,3204000.0


<h3> 5. Xét chi trả Bonus theo doanh số quý:

In [134]:
quarterly_bonus = pd.merge(fyp_quarter, com_final, left_on ='Servicing Agent', right_on = 'Commission Agent', how ='left')
quarterly_bonus = quarterly_bonus[['Servicing Agent', 'FYP_Final', 'FYC']]
quarterly_bonus['%FYC_Bonus'] = np.where(quarterly_bonus['FYP_Final'] >= 40000000, 0.05, 
                            np.where(quarterly_bonus['FYP_Final'] >= 80000000, 0.1,  
                            np.where(quarterly_bonus['FYP_Final'] >= 100000000, 0.15,
                            np.where(quarterly_bonus['FYP_Final'] >= 125000000, 0.2, 0))))
quarterly_bonus                                        

,Servicing Agent,FYP_Final,FYC,%FYC_Bonus
0,60000194,0.0,0.0,0.0
1,60000288,0.0,0.0,0.0
2,60000321,0.0,0.0,0.0
3,60000449,714100.0,0.0,0.0
4,60000450,0.0,0.0,0.0
...,...,...,...,...
3489,60083617,4563000.0,1369000.0,0.0
3490,69999899,0.0,NaN,0.0
3491,69999997,1409400.0,NaN,0.0
3492,69999998,10978600.0,NaN,0.0


In [136]:
quarterly_bonus['Quarterly Bonus'] = quarterly_bonus['FYC'] * quarterly_bonus['%FYC_Bonus']
quarterly_bonus.sort_values('Quarterly Bonus', ascending = False)

,Servicing Agent,FYP_Final,FYC,%FYC_Bonus,Quarterly Bonus
6,60000482,40589000.0,8597000.0,0.05,429850.0
0,60000194,0.0,0.0,0.00,0.0
2330,60042839,174600.0,10000.0,0.00,0.0
2319,60042670,0.0,0.0,0.00,0.0
2320,60042720,0.0,0.0,0.00,0.0
...,...,...,...,...,...
3219,60068923,-490000.0,NaN,0.00,NaN
3490,69999899,0.0,NaN,0.00,NaN
3491,69999997,1409400.0,NaN,0.00,NaN
3492,69999998,10978600.0,NaN,0.00,NaN
